# Combined Client Example

This notebook demonstrates how to use the combined `LangGateLocal` client, which provides access to both registry and parameter transformation functionality in a single client.

In [2]:
import os
from pprint import pprint as pp

# Optional environment variables
# os.environ["LANGGATE_CONFIG"] = "path/to/langgate_config.yaml"
# os.environ["LANGGATE_MODELS"] = "path/to/langgate_models.json"
# os.environ["LOG_LEVEL"] = "debug"

## Initialize the Combined Client

In [ ]:
from langgate.sdk import LangGateLocal

# Initialize the combined client
# LangGateLocal is a singleton that combines both registry and transformer functionality
client = LangGateLocal()

## List Available Models

In [4]:
# List available models
models = await client.list_models()
print(f"Available models: {len(models)}")
for model in models[:5]:
    print(f"- {model.id}: {model.name}")

2025-04-05 21:03:16 [debug    ] refreshed_model_cache          model_count=5
Available models: 5
- openai/gpt-4o: GPT-4o
- openai/o1: o1
- openai/o1-high: o1-high
- anthropic/claude-3-7-sonnet: Claude-3.7 Sonnet
- anthropic/claude-3-7-sonnet-reasoning: Claude-3.7 Sonnet R


## Define Input Parameters

In [5]:
# Sample user parameters
input_params = {
    "temperature": 0.7,
    "max_tokens": 1000,
    "stream": True,
}

## Get Model Info and Transform Parameters

Using the combined client, we can get model information and transform parameters from a unified interface.

In [7]:
if models:
    model_id = models[0].id

    # Get model info
    model_info = await client.get_model_info(model_id)
    print(f"Model: {model_info.name}")
    print(f"Provider: {model_info.provider.name}")
    print(f"Description: {model_info.description}")

    # Transform parameters
    transformed = await client.get_params(model_id, input_params)
    print("\nTransformed parameters:")
    pp(transformed)

Model: GPT-4o
Provider: OpenAI
Description: The GPT-4o (omni) model from OpenAI builds upon the GPT-4 series with improved performance and multimodal capabilities. GPT-4o is great for most tasks.

Transformed parameters:
{'api_key': SecretStr('**********'),
 'base_url': 'https://api.openai.com/v1',
 'max_tokens': 1000,
 'model': 'gpt-4o',
 'stream': True,
 'temperature': 0.7}


## Working with Model-Specific Configurations

Different models may have different parameter transformations defined in `langgate_config.yaml`.
You can define "virtual models" with IDs referring to a model + a configuration.

In [11]:
# Using a model with a specific configuration.
# Anthropic has no actual model with this ID. We define it ourselves,
# to reference claude-3-7-sonnet with a specific reasoning configuration.
model_id = "anthropic/claude-3-7-sonnet-reasoning"

# Get model info
model_info = await client.get_model_info(model_id)
print(f"Model: {model_info.name}")
print(f"Provider: {model_info.provider.name}")
print(f"Description: {model_info.description}")

# Transform parameters
transformed = await client.get_params(model_id, input_params)
print("\nTransformed parameters:")
pp(transformed)

Model: Claude-3.7 Sonnet R
Provider: Anthropic
Description: Claude-3.7 Sonnet with reasoning capabilities.

Transformed parameters:
{'api_key': SecretStr('**********'),
 'base_url': 'https://api.anthropic.com',
 'max_tokens': 1000,
 'model': 'claude-3-7-sonnet-20250219',
 'stream': True,
 'thinking': {'budget_tokens': 1024, 'type': 'enabled'}}


## Working with Model Capabilities

In [12]:
# Check model capabilities
if models:
    model_id = models[0].id
    model_info = await client.get_model_info(model_id)

    print(f"Capabilities for {model_info.name}:")
    pp(model_info.capabilities.model_dump(exclude_none=True))

    # Adjust parameters based on capabilities
    custom_params = input_params.copy()

    # Example: Add tools if supported
    if model_info.capabilities.supports_tools:
        custom_params["tools"] = [
            {
                "type": "function",
                "function": {
                    "name": "get_weather",
                    "description": "Get the current weather in a location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and country, e.g., Cork, Ireland",
                            }
                        },
                        "required": ["location"],
                    },
                },
            }
        ]

    # Transform the custom parameters
    transformed = await client.get_params(model_id, custom_params)
    print("\nTransformed parameters with tools:")
    pp(transformed)

Capabilities for GPT-4o:
{'supports_parallel_tool_calls': True,
 'supports_prompt_caching': True,
 'supports_response_schema': True,
 'supports_system_messages': True,
 'supports_tool_choice': True,
 'supports_tools': True,
 'supports_vision': True}

Transformed parameters with tools:
{'api_key': SecretStr('**********'),
 'base_url': 'https://api.openai.com/v1',
 'max_tokens': 1000,
 'model': 'gpt-4o',
 'stream': True,
 'temperature': 0.7,
 'tools': [{'function': {'description': 'Get the current weather in a location',
                         'name': 'get_weather',
                         'parameters': {'properties': {'location': {'description': 'The '
                                                                                   'city '
                                                                                   'and '
                                                                                   'country, '
                                                             